In [1]:
import chromadb
import numpy as np
import pandas as pd

# Vector DB - Chroma DB 구축

In [20]:
df = pd.read_csv('날짜별_의정활동.csv')

In [7]:
client = chromadb.PersistentClient(path="../data")
collection = client.get_or_create_collection(
    name="legal_info",
    metadata={"hnsw:space": "cosine"}
)

In [21]:
# 인덱스
ids = []
# 메타 데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장한다고 한다. 
documents = []

In [22]:
for idx in range(len(df)):
    # iloc: index 위치를 기반으로 df에서 행을 선택
    item = df.iloc[idx]
    id = item['의안명'].replace(' ', '-')
    document = f"(의안명: {item['의안명']}) (의안번호: {item['의안번호']}) - 단계:{item['단계']}, 세부단계: {item['세부단계']}, 소관위원회: {item['소관위원회']}"
    meta = {
        "발의 일자": item['일자']
    }
    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

# DB 저장

In [23]:
collection.add(
    documents = documents,
    metadatas=doc_meta,
    ids=ids
)

/home/jisoo/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|████████████████| 79.3M/79.3M [00:12<00:00, 6.90MiB/s]


# DB 쿼리

In [38]:
collection.query(
    query_texts=["경찰공무원법"],
    n_results=5,
    where={"발의 일자":"2023-11-23"},
)

ValueError: Expected where value to be a str, int, float, or operator expression, got ['BETWEEN', '2023-11-01', 'AND', '2023-11-23']